In [18]:
import pandas as pd
import seaborn as sb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score


# merging two csv files
#flights = pd.concat(
#    map(pd.read_csv, ["../BITS PILANI FINAL PROJECT/2007.csv", "../BITS PILANI FINAL PROJECT/2008.csv"]), ignore_index=True)

df=pd.read_csv("../BITS PILANI FINAL PROJECT/2007.csv")
df

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,...,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,2007,1,1,1,1232.0,1225,1341.0,1340,WN,2891,...,4,11,0,NaN,0,0,0,0,0,0
1,2007,1,1,1,1918.0,1905,2043.0,2035,WN,462,...,5,6,0,NaN,0,0,0,0,0,0
2,2007,1,1,1,2206.0,2130,2334.0,2300,WN,1229,...,6,9,0,NaN,0,3,0,0,0,31
3,2007,1,1,1,1230.0,1200,1356.0,1330,WN,1355,...,3,8,0,NaN,0,23,0,0,0,3
4,2007,1,1,1,831.0,830,957.0,1000,WN,2278,...,3,9,0,NaN,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7453210,2007,12,15,6,1558.0,1605,1749.0,1736,DL,58,...,14,17,0,NaN,0,0,0,0,0,0
7453211,2007,12,15,6,1902.0,1851,2110.0,2105,DL,59,...,6,21,0,NaN,0,0,0,0,0,0
7453212,2007,12,15,6,1024.0,1025,1750.0,1735,DL,61,...,14,19,0,NaN,0,0,0,15,0,0
7453213,2007,12,15,6,1353.0,1315,1658.0,1622,DL,62,...,11,14,0,NaN,0,0,0,0,0,36


In [19]:
#flights_needed_data = flights[0:100000]  # getting a segment 
#flights_needed_data = flights
#flights_needed_data.info()  # for an insight into the data
#df=df[0:1000000]
df=df[0:500000]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 29 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Year               500000 non-null  int64  
 1   Month              500000 non-null  int64  
 2   DayofMonth         500000 non-null  int64  
 3   DayOfWeek          500000 non-null  int64  
 4   DepTime            487243 non-null  float64
 5   CRSDepTime         500000 non-null  int64  
 6   ArrTime            486315 non-null  float64
 7   CRSArrTime         500000 non-null  int64  
 8   UniqueCarrier      500000 non-null  object 
 9   FlightNum          500000 non-null  int64  
 10  TailNum            500000 non-null  object 
 11  ActualElapsedTime  486315 non-null  float64
 12  CRSElapsedTime     499967 non-null  float64
 13  AirTime            486315 non-null  float64
 14  ArrDelay           486315 non-null  float64
 15  DepDelay           487243 non-null  float64
 16  Or

In [20]:
# filtering out unnecessary columns
#flights_needed_data=flights_needed_data.drop(['Year','Month','DayofMonth','CRSArrTime','CRSElapsedTime','CRSDepTime','FlightNum','TailNum',
#                                              'FlightNum','Distance','TaxiOut','TaxiIn',
#                                              'CRSArrTime','CRSDepTime','CancellationCode'], axis=1)

#flights_needed_data


df=df.drop(['Year','Month','DayofMonth','CRSArrTime','CRSElapsedTime','CRSDepTime','FlightNum','TailNum',
                                              'FlightNum','Distance','TaxiOut','TaxiIn',
                                              'CRSArrTime','CRSDepTime','CancellationCode'], axis=1)

df

,DayOfWeek,DepTime,ArrTime,UniqueCarrier,ActualElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Cancelled,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,1,1232.0,1341.0,WN,69.0,54.0,1.0,7.0,SMF,ONT,0,0,0,0,0,0,0
1,1,1918.0,2043.0,WN,85.0,74.0,8.0,13.0,SMF,PDX,0,0,0,0,0,0,0
2,1,2206.0,2334.0,WN,88.0,73.0,34.0,36.0,SMF,PDX,0,0,3,0,0,0,31
3,1,1230.0,1356.0,WN,86.0,75.0,26.0,30.0,SMF,PDX,0,0,23,0,0,0,3
4,1,831.0,957.0,WN,86.0,74.0,-3.0,1.0,SMF,PDX,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,3,1342.0,1718.0,AA,156.0,121.0,18.0,2.0,ORD,BOS,0,0,0,0,18,0,0
499996,4,1338.0,1655.0,AA,137.0,103.0,-5.0,-2.0,ORD,BOS,0,0,0,0,0,0,0
499997,5,1440.0,1741.0,AA,121.0,103.0,41.0,60.0,ORD,BOS,0,0,0,0,2,0,39
499998,6,1342.0,1701.0,AA,139.0,104.0,1.0,2.0,ORD,BOS,0,0,0,0,0,0,0


In [21]:
# replacing all NaN values with the mean of the attribute in which they are present
#flights_needed_data=flights_needed_data.fillna(flights_needed_data.mean())

df=df.fillna(df.mean())

In [23]:
print('After fillna Update')

After fillna Update


In [24]:
#df =flights_needed_data
    
for dataset in df:
    df.loc[df['ArrDelay'] <= 15, 'Status'] = 0
    df.loc[df['ArrDelay'] >= 15, 'Status'] = 1
    df.loc[df['ArrDelay'] >= 60, 'Status'] = 1
    df.loc[df['Diverted'] == 1, 'Status'] = 1
    df.loc[df['Cancelled'] == 1, 'Status'] = 1

    # replacing na values in college with No college
df["LateAircraftDelay"].fillna(0, inplace = True)
df["SecurityDelay"].fillna(0, inplace = True)
df["NASDelay"].fillna(0, inplace = True)
df["WeatherDelay"].fillna(0, inplace = True)
df["CarrierDelay"].fillna(0, inplace = True)
#print("Y variable in two categories")

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 18 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   DayOfWeek          500000 non-null  int64  
 1   DepTime            500000 non-null  float64
 2   ArrTime            500000 non-null  float64
 3   UniqueCarrier      500000 non-null  object 
 4   ActualElapsedTime  500000 non-null  float64
 5   AirTime            500000 non-null  float64
 6   ArrDelay           500000 non-null  float64
 7   DepDelay           500000 non-null  float64
 8   Origin             500000 non-null  object 
 9   Dest               500000 non-null  object 
 10  Cancelled          500000 non-null  int64  
 11  Diverted           500000 non-null  int64  
 12  CarrierDelay       500000 non-null  int64  
 13  WeatherDelay       500000 non-null  int64  
 14  NASDelay           500000 non-null  int64  
 15  SecurityDelay      500000 non-null  int64  
 16  La

In [26]:
# removing some more columns
df=df.drop(['Origin', 'Dest', 'UniqueCarrier','ArrDelay'],axis=1)
df


,DayOfWeek,DepTime,ArrTime,ActualElapsedTime,AirTime,DepDelay,Cancelled,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,Status
0,1,1232.0,1341.0,69.0,54.0,7.0,0,0,0,0,0,0,0,0.0
1,1,1918.0,2043.0,85.0,74.0,13.0,0,0,0,0,0,0,0,0.0
2,1,2206.0,2334.0,88.0,73.0,36.0,0,0,3,0,0,0,31,1.0
3,1,1230.0,1356.0,86.0,75.0,30.0,0,0,23,0,0,0,3,1.0
4,1,831.0,957.0,86.0,74.0,1.0,0,0,0,0,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,3,1342.0,1718.0,156.0,121.0,2.0,0,0,0,0,18,0,0,1.0
499996,4,1338.0,1655.0,137.0,103.0,-2.0,0,0,0,0,0,0,0,0.0
499997,5,1440.0,1741.0,121.0,103.0,60.0,0,0,0,0,2,0,39,1.0
499998,6,1342.0,1701.0,139.0,104.0,2.0,0,0,0,0,0,0,0,0.0


In [27]:
df["Status"]=df["Status"].astype("category")
data = df.values

X, y = data[:,:-1], data[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)  # splitting in the ratio 70:30
print("Before decision tree classifer")

Before decision tree classifer


In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 14 columns):
 #   Column             Non-Null Count   Dtype   
---  ------             --------------   -----   
 0   DayOfWeek          500000 non-null  int64   
 1   DepTime            500000 non-null  float64 
 2   ArrTime            500000 non-null  float64 
 3   ActualElapsedTime  500000 non-null  float64 
 4   AirTime            500000 non-null  float64 
 5   DepDelay           500000 non-null  float64 
 6   Cancelled          500000 non-null  int64   
 7   Diverted           500000 non-null  int64   
 8   CarrierDelay       500000 non-null  int64   
 9   WeatherDelay       500000 non-null  int64   
 10  NASDelay           500000 non-null  int64   
 11  SecurityDelay      500000 non-null  int64   
 12  LateAircraftDelay  500000 non-null  int64   
 13  Status             500000 non-null  category
dtypes: category(1), float64(5), int64(8)
memory usage: 50.1 MB


In [29]:
import numpy as np

clf = DecisionTreeClassifier()
clf = clf.fit(X_train,y_train)

pred_prob = clf.predict_proba(X_test)

#n=pred_prob.shape[0]

#print(y_test)
#arr = np.array(n*[y_test])

#print(arr)

print(pred_prob[:,1])

#auc_score = roc_auc_score(arr, pred_prob[:,1])
#auc_score

[0. 0. 0. ... 1. 0. 0.]


In [30]:
print(y_test)

[0. 0. 0. ... 1. 0. 0.]


In [31]:
auc_score = roc_auc_score(y_test, pred_prob[:,1])

In [32]:
auc_score

0.9999397851450625